In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import csv

In [2]:
# Set the User-Agent header
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36',
           'Accept-Language': 'en-US, en;q=0.5'})

In [3]:
# Initialize lists to store data
movie_name = []
year = []
time = []
rating = []
metascore = []
director = []
votes = []
gross = []
description = []
genre = []
cast = []

pages = np.arange(1, 1000, 50)

In [4]:
# Iterate through pages
for page in pages:
    page = requests.get("https://www.imdb.com/search/title/?title_type=feature&primary_language=en&start=" + str(page) + "&ref_=adv_nxt", headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    movie_data = soup.findAll('div', attrs={'class': 'lister-item mode-advanced'})
    
    # Iterate through movie data on the page
    for store in movie_data:
        name = store.h3.a.text
        movie_name.append(name)

        year_of_release = store.h3.find('span', class_="lister-item-year text-muted unbold").text.strip('()')
        year.append(year_of_release)

        runtime = store.p.find("span", class_='runtime').text if store.find('span', class_="runtime") else "NA"
        time.append(runtime)

        gen = store.p.find("span", class_='genre').text
        genre.append(gen)

        rate = store.find('div', class_="inline-block ratings-imdb-rating").text.replace('\n', '').strip() if store.find('div', class_="inline-block ratings-imdb-rating") else "NA"
        rating.append(rate)

        meta = store.find('span', class_="metascore").text if store.find('span', class_="metascore") else "NA"
        metascore.append(meta)

        dire = store.find('p', class_='').find_all('a')[0].text
        director.append(dire)

        # Extract cast information
        cast_links = store.find('p', class_='').find_all('a')[1:]
        cast_names = [a.text for a in cast_links]
        cast.append(cast_names)

        value = store.find_all('span', attrs={'name': 'nv'})
        vote = value[0].text if value else 'NA'
        votes.append(vote)

        describe = store.find_all('p', class_='text-muted')
        description_ = describe[1].text.replace('\n', '').strip() if len(describe) > 1 else 'NA'
        description.append(description_)


In [5]:
# Create DataFrame
movie_list = pd.DataFrame({
    "Movie Name": movie_name,
    "Year of Release": year,
    "Watch Time": time,
    "Genre": genre,
    "Movie Rating": rating,
    "Metascore of movie": metascore,
    "Director": director,
    "Cast": [','.join(c) for c in cast],  # Convert list of cast names to comma-separated string
    "Votes": votes,
    "Description": description
})

In [6]:
# Export to CSV
movie_list.to_csv("movie_data.csv", index=False)

In [4]:
df = pd.read_csv('movie_data.csv')
df.head()

,Movie Name,Year of Release,Watch Time,Genre,Movie Rating,Metascore of movie,Director,Cast,Votes,Description
0,Oppenheimer,2023,180 min,"\nBiography, Drama, History",8.6,88.0,Christopher Nolan,"Cillian Murphy,Emily Blunt,Matt Damon,Robert D...","368,392","The story of American scientist, J. Robert Opp..."
1,Barbie,2023,114 min,"\nAdventure, Comedy, Fantasy",7.4,80.0,Greta Gerwig,"Margot Robbie,Ryan Gosling,Issa Rae,Kate McKinnon","246,653",Barbie suffers a crisis that leads her to ques...
2,Heart of Stone,2023,122 min,"\nAction, Crime, Thriller",5.7,44.0,Tom Harper,"Gal Gadot,Jamie Dornan,Alia Bhatt,Jing Lusi","29,166",An intelligence operative for a shadowy global...
3,"Red, White & Royal Blue",2023,118 min,"\nComedy, Romance",7.1,62.0,Matthew López,"Taylor Zakhar Perez,Nicholas Galitzine,Uma Thu...","22,778",When the feud between the son of the American ...
4,Blue Beetle,2023,127 min,"\nAction, Adventure, Sci-Fi",6.8,61.0,Angel Manuel Soto,"Xolo Maridueña,Bruna Marquezine,Becky G,Damián...","11,680",An alien scarab chooses college graduate Jaime...
